# Project Euler
## Problems 91 - 100

### ************************

## Problem #91

How many right triangles can be formed with the origin where 0 <= x1, x2, y1, y2 <= 50?

In [7]:
def rightTrianglesWithin(cap):
    points = []
    total = 0
    for i in range(cap + 1):
        for j in range(cap + 1):
            points.append((i,j))
    for i in range(1, len(points)-1):
        for j in range(i + 1, len(points)):
            if isRightTriangle(points[i], points[j]):
                total += 1
    return total

In [1]:
def isRightTriangle(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    if y1 * x2 == y2 * x1:
        return False
    lengths = sorted([x1**2 + y1**2, x2**2 + y2**2, (x1-x2)**2 + (y1-y2)**2])
    return lengths[0] + lengths[1] == lengths[2]

In [12]:
rightTrianglesWithin(50)

14234

## Problem #92

How many starting numbers below ten million will arrive at 89 by adding the squares of the digits?

In [15]:
def eightyNineChains(cap):
    memo = {1:1, 89:89}
    sq = {x:x**2 for x in range(10)}
    total = 0
    for i in range(2, cap):
        if digitSquareChain(i, sq, memo) == 89:
            total += 1
    return total

In [14]:
def digitSquareChain(n, sq, memo):
    if n in memo:
        return memo[n]
    dss = digitSquareSum(n, sq)
    if dss in memo:
        return memo[dss]
    memo[dss] = digitSquareChain(dss, sq, memo)
    return memo[dss]

In [13]:
def digitSquareSum(n, sq):
    total = 0
    while n > 0:
        n, r = divmod(n, 10)
        total += sq[r]
    return total

In [21]:
eightyNineChains(10000000)

8581146

## Problem #93

Find the set of four distinct digits a < b < c < d for which the longest set of consecutive positive integers 1 to n can be obtained, giving your answer as a string: abcd.

In [4]:
from itertools import product
from itertools import permutations
from itertools import combinations

In [15]:
def findBestFourDigits():
    bestLength = 0
    bestFour = None
    for digs in combinations(range(10), 4):
        ies = integerExpressionSet(digs)
        n = 1
        while n in ies:
            n += 1
        if n-1 > bestLength:
            bestFour = digs
            bestLength = n-1
    print('Longest chain:', bestLength)
    return ''.join([str(x) for x in bestFour])

In [6]:
def integerExpressionSet(digits):
    ints = set()
    for perm in permutations(digits, 4):
        nums = []
        for p in perm:
            nums.append([p, 1])
        for ops in product(range(6), repeat=3):
            val = nums[0]
            for i, op in enumerate(ops):
                val = doop(op, val, nums[i+1])
            val = reduceFrac(val)
            if val[1] == 1 and val[0] > 0:
                ints.add(val[0])
    return ints        

In [7]:
def doop(op, frac1, frac2):
    if op == 0:
        return addFrac(frac1, frac2)
    elif op == 1:
        frac3 = [frac2[0]*-1, frac2[1]]
        return addFrac(frac1, frac3)
    elif op == 2:
        return multFrac(frac1, frac2)
    elif op == 3:
        frac3 = [frac2[1], frac2[0]]
        return multFrac(frac1, frac3)
    elif op == 4:
        frac3 = [frac1[0]*-1, frac1[1]]
        return addFrac(frac2, frac3)
    else:
        frac3 = [frac1[1], frac1[0]]
        return multFrac(frac2, frac3)

In [8]:
def multFrac(frac1, frac2):
    a, b = frac1
    c, d = frac2
    return [a*c, b*d]

In [9]:
def addFrac(frac1, frac2):
    a, b = frac1
    c, d = frac2
    return [b*c + a*d, b*d]

In [10]:
def reduceFrac(frac):
    a, b = frac
    if b == 0:
        return [1, 0]
    d = gcd(a, b)
    return [a // d, b // d]

In [11]:
def gcd(a, b):
    if b == 0:
        return a
    return gcd(b, a % b)

In [16]:
findBestFourDigits()

Longest chain: 51


'1258'

## Problem #94

Find the sum of the perimeters of all almost equilateral triangles with integral side lengths and area and whose perimeters do not exceed one billion.

In [127]:
import math

In [138]:
def almostEquilateralTris(cap):
    k = 2
    alms = []
    total = 0
    while 6 * k + 2 < cap:
        for a in [2*k-1,2*k+1]:
            b = math.floor((a**2 - k**2)**0.5)
            if k**2 + b**2 == a**2:
                alms.append((k,b,a))
                total += 2*a + 2*k
        k += 1
    print(alms)
    return total

In [145]:
almostEquilateralTris(1000000000)

[(3, 4, 5), (8, 15, 17), (33, 56, 65), (120, 209, 241), (451, 780, 901), (1680, 2911, 3361), (6273, 10864, 12545), (23408, 40545, 46817), (87363, 151316, 174725), (326040, 564719, 652081), (1216801, 2107560, 2433601), (4541160, 7865521, 9082321), (16947843, 29354524, 33895685), (63250208, 109552575, 126500417)]


518408346

## Problem #95

Find the smallest member of the longest amicable chain with no element exceeding one million.

In [28]:
import math

In [54]:
def smallestLongestAmicable(cap):
    primes = generatePrimesTo(math.ceil(cap**0.5))
    memo = {1:0}
    chains = dict()
    mxChain = 0
    mxNum = 1
    for n in range(2, cap+1):
        curSet, nums = set(), list()
        chLength = amicableChain(n, primes, memo, chains, cap, curSet, nums)
        if chLength > mxChain:
            mxChain = chLength
            mxNum = n
    print("Max Chain Length:", mxChain)
    return mxNum

In [29]:
def amicableChain(n, primes, memo, chains, cap, curSet, nums):
    if n in memo:
        val = memo[n]
        if len(nums) > 0:
            for num in nums:
                memo[num] = 0
        return val
    if n > cap:
        for val in nums:
            memo[val] = 0
        return 0
    if n in curSet:
        i = 0
        while nums[i] != n:
            i += 1
        length = len(nums) - i
        mn = n
        for j in range(i):
            memo[nums[j]] = 0
        for j in range(i, len(nums)):
            val = nums[j]
            memo[val] = length
            if val < mn:
                mn = val
        if length in chains:
            if mn < chains[length]:
                chains[length] = mn
        else:
            chains[length] = mn
        return length
    curSet.add(n)
    nums.append(n)
    pds = properDivSum(n, primes)
    amicableChain(pds, primes, memo, chains, cap, curSet, nums)
    return memo[n]

In [24]:
def properDivSum(n, primes):
    divs = 1
    n_copy = n
    root = n**0.5
    for p in primes:
        if n == 1 or root < p:
            break
        power = 1
        curSum = 1
        while n % p == 0:
            n //= p
            power *= p
            curSum += power
        divs *= curSum
    if n != 1:
        divs *= (n+1)
    return divs - n_copy

In [25]:
def generatePrimesTo(n):
    primes = [2,3,5,7]
    i = 11
    while i < n:
        for j in [0, 2, 6, 8]:
            if isPrime(i+j, primes):
                primes.append(i+j)
        i += 10
    while primes[-1] > n:
        primes.pop()
    return primes

In [26]:
def isPrime(n, primes):
    if n < 11:
        return False if n not in [2,3,5,7] else True
    i = 0
    root = math.floor(n**0.5)
    while i < len(primes) and primes[i] <= root:
        if n % primes[i] == 0:
            return False
        i += 1
    return True

In [55]:
smallestLongestAmicable(1000000)

Max Chain Length: 28


14316

## Problem #96

By solving all fifty puzzles find the sum of the 3-digit numbers found in the top left corner of each solution grid; for example, 483 is the 3-digit number found in the top left corner of the solution grid above.

In [377]:
def solveAllGrids():
    total = 0
    for n in range(50):
        sud = readSudoku('sudoku_p96.txt')[9*n:9*(n+1)]
        pmap = dict()
        for i in range(9):
            for j in range(9):
                pmap[(i,j)] = set(range(1,10))
        stack = getSudokuStack(sud, pmap)
        result = backTrackSudoku(sud, pmap, stack)
        total += 100*result[0][0] + 10*result[0][1] + result[0][2]
    return total

In [367]:
def backTrackSudoku(sud, pmap, stack):
    if len(stack) == 0:
        for i in range(9):
            for j in range(9):
                if sud[i][j] == 0:
                    return None
        return sud
    k, coord = stack.pop()
    if len(pmap[coord]) == 0:
        stack.append([k, coord])
        return None
    r, c = coord
    oldSet = set(pmap[coord])
    for val in oldSet:
        sud[r][c] = val
        newCoords = updateGrid(val, coord, pmap)
        bt = backTrackSudoku(sud, pmap, stack)
        if bt is not None:
            return bt
        for pos in newCoords:
            pmap[pos].add(val)
    pmap[coord] = oldSet
    sud[r][c] = 0
    stack.append([k, coord])
    return None

In [351]:
def updateGrid(val, coord, pmap):
    updatedCoords = []
    pmap[coord] = set()
    r, c = coord
    for i in range(1, 9):
        for pos in [(r, (c + i)%9), ((r + i)%9, c)]:
            if val in pmap[pos]:
                updatedCoords.append(pos)
                pmap[pos].remove(val)
    i, j = (r // 3)*3, (c // 3)*3
    for a in range(3):
        for b in range(3):
            pos = (i+a, j+b)
            if pos == coord:
                continue
            if val in pmap[pos]:
                updatedCoords.append(pos)
                pmap[pos].remove(val)
    return updatedCoords

In [292]:
def getSudokuStack(sud, pmap):
    stack = []
    for r in range(9):
        for c in range(9):
            val = sud[r][c]
            coord = (r,c)
            if val == 0:
                stack.append(coord)
                continue
            updateGrid(val, coord, pmap)
    newStack = [[len(pmap[pos]), pos] for pos in stack]
    newStack.sort(reverse=True)
    return newStack

In [294]:
def readSudoku(filename):
    sud = []
    with open(filename, 'r') as f:
        for line in f.readlines():
            line = line.strip()
            if line.startswith('G'):
                continue
            sud.append([int(x) for x in line])
    return sud

In [380]:
solveAllGrids()

24702

## Problem #97

In [58]:
def largeNonMersennePrime():
    num = 28433
    k = 7830457
    ten = 10**10
    for _ in range(k):
        num *= 2
        num %= ten
    return (num + 1) % ten

In [59]:
largeNonMersennePrime()

8739992577

## Problem #98

Find all the square anagram word pairs (a palindromic word is NOT considered to be an anagram of itself). What is the largest square number formed by any member of such a pair?

In [88]:
import math

In [118]:
def largestAnagramicSquare():
    grams = findAnagramWords(readWords('anagramicSquares_p98.txt'))
    mxSquare = -1
    mxPair = ["", ""]
    for w1, w2 in grams:
        val = squareForPair(w1, w2)
        if val > mxSquare:
            mxSquare = val
            mxPair = [w1, w2]
    print(mxPair)
    return mxSquare

In [109]:
def squareForPair(w1, w2):
    n = len(w1) - 1
    squares = set()
    i = math.ceil((10**n)**0.5)
    cap = math.ceil((10**(n+1))**0.5)
    mxNum = -1
    while i < cap:
        squares.add(i**2)
        i += 1
    for sq in squares:
        digs = digList(sq)
        letVal = dict()
        distincts = set()
        isGood = True
        for char, d in zip(w1, digs):
            if char in letVal:
                if d != letVal[char]:
                    isGood = False
                    break
            else:
                if d in distincts:
                    isGood = False
                    break
                letVal[char] = d
                distincts.add(d)
        if isGood:
            otherDigs = [letVal[char] for char in w2]
            num = digsToNum(otherDigs)
            if num in squares:
                num = max(sq, num)
                if num > mxNum:
                    mxNum = num
    return mxNum

In [102]:
def digsToNum(digs):
    num = digs[0]
    for i in range(1, len(digs)):
        num *= 10
        num += digs[i]
    return num

In [100]:
def digList(num):
    digs = []
    while num > 0:
        num, r = divmod(num, 10)
        digs.append(r)
    return digs[::-1]

In [84]:
def findAnagramWords(words):
    letterValues = dict()
    letters = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    primes = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101]
    wordValues = dict()
    anagramList = list()
    for l, p in zip(letters, primes):
        letterValues[l] = p
    for word in words:
        val = 1
        for char in word:
            val *= letterValues[char]
        if val in wordValues:
            wordValues[val].append(word)
        else:
            wordValues[val] = [word]
    for x in wordValues.values():
        if len(x) == 2:
            anagramList.append(x)
        if len(x) > 2:
            for i in range(len(x)-1):
                for j in range(i+1, len(x)):
                    anagramList.append([x[i], x[j]])
    return anagramList

In [85]:
def readWords(filename):
    with open(filename, 'r') as f:
        words = f.readlines()[0].split('\",\"')
        words[0] = words[0].strip('\"')
        words[-1] = words[-1].strip('\"')
    return words

In [119]:
largestAnagramicSquare()

['BOARD', 'BROAD']


18769

## Problem #99

Using the text file containing one thousand lines with a base/exponent pair on each line, determine which line number has the greatest numerical value.

In [122]:
import math

In [123]:
def greatestExponentialValue():
    pairs = readExponentials('largestExponential_p99.txt')
    mxVal = -1
    mxPos = -1
    for i, pair in enumerate(pairs):
        pos = i + 1
        a, b = pair
        a = math.log10(a)
        val = a * b
        if val > mxVal:
            mxVal = val
            mxPos = pos
    return mxPos

In [120]:
def readExponentials(filename):
    pairs = []
    with open(filename, 'r') as f:
        for line in f.readlines():
            line = [int(x) for x in line.strip().split(',')]
            pairs.append(line)
    return pairs

In [124]:
greatestExponentialValue()

709

## Problem #100

By finding the first arrangement to contain over one trillion discs in total, determine the number of blue discs that the box would contain.

In [161]:
import math

In [198]:
def firstArrangedProbabilityOver(cap):
    rn, rd = 3,2
    B,T = 3,4
    i = 0
    while True:
        nrn, nrd = rn + 2*rd, rn + rd
        if i % 2 == 0:
            B = rn * nrd
            T = rn * nrn
        else:
            B = rd * nrn
            T = 2 * rd * nrd
        if T > cap:
            print("Total Discs:", T)
            return B
        rn, rd = nrn, nrd
        i += 1

In [199]:
firstArrangedProbabilityOver(10**12)

Total Discs: 1070379110497


756872327473